In [17]:
import json

with open("/scr/ig_pipeline/metadata/fillable_assignments.json") as f:
    data = json.load(f)

In [2]:
import collections

ctr = collections.Counter(data.values())
print(ctr.most_common())

has_assignment = {k for k, v in data.items() if v in {"combined", "ray", "dip"}}

[('combined', 268), ('ray', 156), ('fix', 64), ('manual', 61), ('dip', 30), ('changecollision', 10), ('notfillable', 9), ('fixorn', 2)]


In [3]:
from bddl.knowledge_base import *

Loading BDDL knowledge base... This may take a few seconds.
[nltk_data] Downloading package wordnet to /home/cgokmen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
objects = Object.all_objects()

In [5]:
# Get all the objects that have a fillable synset
fillable_needing = {o.name.split("-")[-1] for o in objects if any(p.name == "fillable" for p in o.category.synset.properties)}

In [6]:
# Get all teh objects that ahve a fillable volume
fillable_annotated = {o.name.split("-")[-1] for o in objects if any(m.name == "fillable" for m in o.meta_links)}

In [7]:
fillable_missing = fillable_needing - fillable_annotated - has_assignment

In [8]:
print("Fillable needing: ", len(fillable_needing))
print("Fillable annotated: ", len(fillable_annotated))
print("Fillable assigned:", len(has_assignment))
print("Fillable missing:", len(fillable_missing))
print("Fillable missing assignment only:", len(fillable_needing - has_assignment))

Fillable needing:  591
Fillable annotated:  458
Fillable assigned: 454
Fillable missing: 57
Fillable missing assignment only: 137


In [9]:
# Are there any fillables that did not get processed?
print({o.name for o in objects if any(p.name == "fillable" for p in o.category.synset.properties) and o.name.split("-")[-1] not in data})

{'trash_can-jlawet', 'soda_cup-fsfsas'}


In [11]:
# Export a copy removing everything with no mesh selection
selected_only = {k: v for k, v in data.items() if v in {"combined", "ray", "dip"}}
with open("/scr/ig_pipeline/metadata/fillable_assignments_selected.json", "w") as f:
    json.dump(selected_only, f, indent=4)

In [12]:
# Group all the to-be-fixed objects by provider
grouped = collections.defaultdict(set)
fix_needing = {k for k, v in data.items() if v == "fix"}
for obj in Object.all_objects():
    model = obj.name.split("-")[-1]
    if model not in fix_needing:
        continue
    grouped[obj.provider].add(obj.name)

for provider, objs in grouped.items():
    print(f"\n{provider}:")
    for obj in sorted(objs):
        print(f"  - {obj}")


objects/batch-11:
  - bird_feeder-hvlkjx
  - tray-glwebh

scenes/house_double_floor_lower:
  - blender-xnjqix
  - vase-jpwsrp
  - washer-ziomqg

scenes/restaurant_brunch:
  - canteen-ouhqnw
  - jar-waousd
  - jar-zdeyzf
  - oil_bottle-gcixra
  - rum_bottle-ueagnt

objects/batch-09:
  - canteen-ttxunv

objects/legacy_batch-10:
  - carafe-ocjcgp
  - coffee_maker-pyttso
  - pitcher-ompiss

scenes/school_computer_lab_and_infirmary:
  - carton-cdmmwy

scenes/school_biology:
  - carton-hhlmbi
  - round_bottom_flask-tqyiso

scenes/school_chemistry:
  - carton-libote
  - petri_dish-vbiqcq
  - reagent_bottle-trtrsl
  - reagent_bottle-ukayce
  - reagent_bottle-xstykf
  - round_bottom_flask-fjytro
  - round_bottom_flask-injdmj

objects/legacy_batch-09:
  - carton-sxlklf

scenes/grocery_store_convenience:
  - carton-ylrxhe
  - paper_cup-guobeq

scenes/house_single_floor:
  - chalice-sfkezf
  - clothes_dryer-jasnzj
  - jar-jdwvyt
  - jar-ociqav
  - jar-pjaljg
  - jar-vxqpnm
  - washer-ynwamu

obje

In [13]:
not_fillable = {k for k, v in data.items() if v in {"notfillable"}}
synset_fillable_marked_notfillable = fillable_needing & not_fillable
print("\n".join(sorted([x.name for x in Object.all_objects() if x.name.split("-")[-1] in synset_fillable_marked_notfillable])))

In [14]:
fixorn = {k for k, v in data.items() if v in {"fixorn"}}
synset_fillable_marked_fixorn = fillable_needing & fixorn
print("\n".join(sorted([x.name for x in Object.all_objects() if x.name.split("-")[-1] in synset_fillable_marked_fixorn])))

bowl-fedafr
saucepot-pkfydm


In [ ]:
# Remove the collision assignments of everything in the fix and changecollision piles
import glob

if False:
    for file in glob.glob("/scr/ig_pipeline/cad/*/*/artifacts/collision_selection.json"):
        # Open each target's collision selection file
        with open(file, "r") as f:
            collision_selections = json.load(f)

        # Find which models are in the fix and changecollision piles
        keys_to_remove = []
        for k, v in collision_selections.items():
            model_id = k.split("-")[1]
            if model_id in data and data[model_id] in {"fix", "changecollision"}:
                keys_to_remove.append(k)

        # Actually remove the collision assignments
        for k in keys_to_remove:
            del collision_selections[k]

        # Write the new collision selection file
        with open(file, "w") as f:
            json.dump(collision_selections, f, indent=4)

In [16]:
# Remove any non-fillable objects from the fillable assignment list
import shutil, glob
if False:
    for obj in objects:
        cat, mdl = obj.name.split("-")
        if mdl not in fillable_needing:
            if mdl in data:
                del data[mdl]
            volume_dirs = glob.glob(f"/scr/ig_pipeline/artifacts/parallels/fillable_volumes/objects/*/{mdl}")
            for vd in volume_dirs:
                shutil.rmtree(vd)

    with open("/scr/ig_pipeline/metadata/fillable_assignments.json", "w") as f:
        json.dump(data, f, indent=4)

In [18]:
# Remove generated meshes for the "fix", "manual", "fixorn" and "changecollision" objects and try again.
import shutil, glob
if False:
    for obj in objects:
        cat, mdl = obj.name.split("-")
        if mdl in data and data[mdl] in {"fix", "manual", "fixorn", "changecollision"}:
            del data[mdl]
            volume_dirs = glob.glob(f"/scr/ig_pipeline/artifacts/parallels/fillable_volumes/objects/*/{mdl}")
            for vd in volume_dirs:
                shutil.rmtree(vd)

    with open("/scr/ig_pipeline/metadata/fillable_assignments.json", "w") as f:
        json.dump(data, f, indent=4)